In [1]:
import copy
import os
import json
from pprint import pp

In [2]:
model_dir = "/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__structured_auto_0.7_1200_1"
out_dir = model_dir.replace("auto", "auto-dedup").replace("solution", "solution-dedup")
out_dir

'/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__structured_auto-dedup_0.7_1200_1'

In [12]:
def new_fingerprint(model_dir, out_dir):
    fingerprint_path = os.path.join(model_dir, "fingerprint.jsonl")
    fingerprint = {}
    with open(fingerprint_path, "r") as f:
        for line in f:
            fingerprint |= json.loads(line)

    fingerprint["n_tool_calls"] += "-dedup"
    new_fingerprint_path = os.path.join(out_dir, "fingerprint.jsonl")

    with open(new_fingerprint_path, "w") as f:
        for key, value in fingerprint.items():
            json_line = json.dumps({key: value})
            f.write(json_line + '\n')

In [21]:
fingerprint_path = os.path.join(model_dir, "fingerprint.jsonl")
fingerprint = {}
with open(fingerprint_path, "r") as f:
    for line in f:
        fingerprint |= json.loads(line)

fingerprint["n_tool_calls"] += "-dedup"
new_fingerprint_path = os.path.join(out_dir, "fingerprint.jsonl")

with open(new_fingerprint_path, "w") as f:
    for key, value in fingerprint.items():
        json_line = json.dumps({key: value})
        f.write(json_line + '\n')

In [3]:
def get_files(path):
    file_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            file_list.append(file_path)
    return file_list

In [4]:
def get_generations(path):
    generations = []
    with open(path, "r") as f:
        for line in f:
            generations.append(json.loads(line))
    return generations

In [5]:
def remove_duplicates(json_list):
    unique_json_list = []
    unique_json_strings = set()

    for json_obj in json_list:
        json_string = json.dumps(json_obj, sort_keys=True)
        if json_string not in unique_json_strings:
            unique_json_list.append(json_obj)
            unique_json_strings.add(json_string)

    return unique_json_list

In [6]:
def bfcl_format(tool_calls):
    tool_strs = []
    for tool_call in tool_calls:
        tool_name = tool_call["tool_name"]
        tool_args = tool_call["tool_arguments"]
        args_string = ', '.join([f"{key}='{value}'" if isinstance(value, str) else f"{key}={value}" for key, value in tool_args.items()])
        tool_str = f'{tool_name}({args_string})'
        tool_strs.append(tool_str)
    result = '[' + ', '.join(tool_strs) + ']'
    return result

In [7]:
def get_new_generations(generations):

    new_generations = []
    _generations = copy.deepcopy(generations)
    for i, gen in enumerate(_generations):
        result = gen["result"]
        if "error" in result:
            new_generations.append(gen)
            continue
        tool_calls = gen["tool_calls"]
        tool_calls = remove_duplicates(tool_calls)
        new_result = bfcl_format(tool_calls)
        gen["result"] = new_result
        gen["tool_calls"] = tool_calls
        new_generations.append(gen)

    return new_generations

In [11]:
def save_new_generations(out_dir, generations_path, new_generations):
    new_generations_path = os.path.join(out_dir, "generations", generations_path.split("/")[-1])
    os.makedirs(os.path.dirname(new_generations_path), exist_ok=True)
    with open(new_generations_path, "w") as f:
        for new_gen in new_generations:
            f.write(json.dumps(new_gen) + "\n")

In [8]:
generations_dir = os.path.join(model_dir, "generations")
generations_paths = get_files(generations_dir)

In [9]:
for generations_path in generations_paths:

    # Load generations
    generations = get_generations(generations_path)

    # Make new generations
    new_generations = get_new_generations(generations)

    # Save new generations
    save_new_generations(out_dir, generations_path, new_generations)

In [22]:
def get_directories(path):
    directories = []

    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            directories.append(item_path)

    return directories

In [23]:
path = "/root/gorilla/berkeley-function-call-leaderboard/outputs"
get_directories(path)

['/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__conditional_solution_0.7_1200',
 '/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__structured_solution_0.7_1200',
 '/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__structured_auto_0.7_1200_1',
 '/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__conditional_auto_0.7_1200_1',
 '/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__unstructured_solution_0.7_1200']

# Explore Results

In [6]:
from tool_use.tool import Tool
from tool_use.prompt import get_system_prompt

In [4]:
# Initialize tool
base_url = "http://localhost:8000/v1"
api_key = "-"
model_name = "databricks/dbrx-instruct"
tool = Tool(base_url, api_key, model_name)

In [7]:
example = {"idx": 38, "result": "[]", "input_token_count": 0, "output_token_count": 0, "latency": 1.2113571166992188, "messages": [{"role": "system", "content": "You are a function calling AI model. Your job is to answer the user's questions and you may call one or more functions to do this.\n\n\n    Please use your own judgment as to whether or not you should call a function. In particular, you must follow these guiding principles:\n    1. You may call one or more functions to assist with the user query. You should call multiple functions when the user asks you to.\n    2. You do not need to call a function. If none of the functions can be used to answer the user's question, please do not make the function call.\n    3. Don't make assumptions about what values to plug into functions. If you are missing the parameters to make a function call, please ask the user for the parameters.\n    4. You may assume the user has implemented the function themselves.\n    5. You may assume the user will call the function on their own. You should NOT ask the user to call the function and let you know the result; they will do this on their own.\n    6. Never call a tool twice with the same exact arguments. Do not repeat your tool calls!\n\n\n    You can only call functions according the following formatting rules:\n    Rule 1: All the functions you have access to are contained within <tool></tools> XML tags. You cannot use any functions that are not listed between these tags.\n\n    Rule 2: For each function call return a json object (using quotes) with function name and arguments within <tool_call>\n{ }\n</tool_call> XML tags as follows:\n    * With arguments:\n    <tool_call>\n{\"tool_name\": \"function_name\", \"tool_arguments\": {\"argument_1_name\": \"value\", \"argument_2_name\": \"value\"} }\n</tool_call>\n    * Without arguments:\n    <tool_call>\n{ \"tool_name\": \"function_name\", \"tool_arguments\": {} }\n</tool_call>\n    In between <tool_call> and</tool_call> tags, you MUST respond in a valid JSON schema.\n    In between the <tool_call> and </tool_call> tags you MUST only write in json; no other text is allowed.\n\n    Rule 3: If user decides to run the function, they will output the result of the function call between the <tool_response> and </tool_response> tags. If it answers the user's question, you should incorporate the output of the function in your answer.\n\n\n    Here are the tools available to you:\n    <tool>\n{\"title\": \"calculate_electrostatic_potential\", \"type\": \"object\", \"description\": \"Calculate the electrostatic potential between two charged bodies using the principle of Coulomb's Law.\", \"properties\": {\"charge1\": {\"type\": \"float\", \"description\": \"The quantity of charge on the first body.\"}, \"charge2\": {\"type\": \"float\", \"description\": \"The quantity of charge on the second body.\"}, \"distance\": {\"type\": \"float\", \"description\": \"The distance between the two bodies.\"}, \"constant\": {\"type\": \"float\", \"description\": \"The value of the electrostatic constant. Default is 8.99e9.\"}}, \"required\": [\"charge1\", \"charge2\", \"distance\"]}\n</tools>\n\n    Remember, don't make assumptions about what values to plug into functions. If you are missing the parameters to make a function call, please ask the user for the parameters. Do not be afraid to ask.\n    "}, {"role": "user", "content": "What is the electrostatic potential between two charged bodies of 1e-9 and 2e-9 of distance 0.05?"}, {"role": "assistant", "content": "Sure, I can help you with that. To calculate the electrostatic potential between two charged bodies, I can use the `calculate_electrostatic_potential` function. However, I need the value of the electrostatic constant to make the function call. Could you please provide me with the value of the electrostatic constant? The default value is 8.99e9.<tool_call>"}], "tool_calls": [], "n_tool_calls": "auto"}
example

{'idx': 38,
 'result': '[]',
 'input_token_count': 0,
 'output_token_count': 0,
 'latency': 1.2113571166992188,
 'messages': [{'role': 'system',
   'content': 'You are a function calling AI model. Your job is to answer the user\'s questions and you may call one or more functions to do this.\n\n\n    Please use your own judgment as to whether or not you should call a function. In particular, you must follow these guiding principles:\n    1. You may call one or more functions to assist with the user query. You should call multiple functions when the user asks you to.\n    2. You do not need to call a function. If none of the functions can be used to answer the user\'s question, please do not make the function call.\n    3. Don\'t make assumptions about what values to plug into functions. If you are missing the parameters to make a function call, please ask the user for the parameters.\n    4. You may assume the user has implemented the function themselves.\n    5. You may assume the user

In [9]:
user_query = example["messages"][1]

{'role': 'user',
 'content': 'What is the electrostatic potential between two charged bodies of 1e-9 and 2e-9 of distance 0.05?'}